In [1]:
import numpy as np
import math
X = np.array([[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]])
Y = np.array([i for i in range(6)]).reshape(-1,1)

In [2]:
class kdNode:
    def __init__(self):
        self.value = None
        self.idx = None
        self.father = None
        self.left = None
        self.right = None

In [3]:
class kdTree:
    def __init__(self,data):
        self.tree = kdNode()
        make_tree(self.tree,data,-1)

# Generate

In [4]:
def make_tree(node,data,idx):
    if len(data)>1:
        N,k = data.shape
        ## 寻找切分坐标轴和切分点
        idx = (idx+1)%k
        median = sorted(data[:,idx])[math.floor(N/2)]
        ## 从数据中找到切分实例
        median_id = list(data[:,idx]).index(median)
        value = data[median_id,:]
        ## 储存
        node.value = value
        node.idx = idx
        ## 切分剩余数据集
        data = np.delete(data,median_id,axis=0)
        left_data = data[data[:,idx]<median,:]
        right_data = data[data[:,idx]>=median,:]
        ## 递归左边和右边
        if len(left_data)!=0:
            node.left = kdNode()
            node.left.father = node
            make_tree(node.left,left_data,idx)
        if len(right_data)!=0:
            node.right = kdNode()
            node.right.father = node
            make_tree(node.right,right_data,idx)
    else:
        node.value = data[0]

In [5]:
tree = kdTree(X)

# Visualize

In [6]:
def print_tree(tree,layer):
    print('\t'*layer,tree.value)
    if tree.left is not None:
        print_tree(tree.left,layer+1)
    if tree.right is not None:
        print_tree(tree.right,layer+1)

print_tree(tree.tree,0)

 [7 2]
	 [5 4]
		 [2 3]
		 [4 7]
	 [9 6]
		 [8 1]


# Find Nearest Neighbor

In [7]:
def L2_dist(x,y):
    return np.sqrt(np.sum((x-y)**2))

def FindBrother(node):
    if node.father.left is node:
        return node.father.right
    if node.father.right is node:
        return node.father.left 

def kdFind(node,x):
    ## 寻找叶节点
    while node.idx is not None:
        if x[node.idx]<node.value[node.idx]:
            if node.left is not None:
                node = node.left
            else:
                break
        else:
            if node.right is not None:
                node = node.right
            else:
                break
    ## 保存当前最近点
    nearest = node.value
    ## 不断向上回退直至根节点
    while node is not None and node.father is not None:
        ## 比较距离
        if L2_dist(node.value,x)<L2_dist(nearest,x):
            nearest = node.value
        ## 是否与兄弟节点相交
        dist = np.abs((node.value-x)[node.father.idx])
        r = L2_dist(nearest,x)
        ## 判断兄弟节点中有无更近的点
        if r>dist:
            try:
                parent_node = FindBrother(node)
                parent_node.father = None
                tmp = kdFind(parent_node,x)
                ## 存在，改动当前最近点
                if L2_dist(tmp,x)<L2_dist(nearest,x):
                    nearest = tmp
                ## 不存在，继续向上递归
                else:
                    node = node.father
                    continue
            except:
                pass
        node = node.father
    return nearest

In [8]:
kdFind(tree.tree,np.array([3,4.5]))

array([2, 3])